In [1]:
import os
import numpy as np
import scipy.stats as st
from tqdm.notebook import trange

from itertools import combinations
import torch.nn.functional as F
import torch

In [2]:
# get currently working directory
base_dir = os.getcwd()

In [3]:
def load_model(model_type='pooling', models_folder='../models'):
    ofile = f'{model_type}_model_1m_20interactions.pt'
    return torch.load(os.path.join(base_dir, models_folder, ofile))

In [4]:
def save_model(model, model_type='pooling', models_folder='../models'):
    ofile = f'{model_type}_model_1m_20interactions.pt'
    return torch.save(model, os.path.join(base_dir, models_folder, ofile))

In [5]:
class StaticVars:
    FLOAT_MAX = np.finfo(np.float32).max
    INT_MAX = np.iinfo(np.int32).max

In [6]:
class InteractionsInfo:
    score = 0
#     interactions = []
#     complete_interactions = []
#     iter_found = -1
    y_loss = 1.0
    proximity_loss = StaticVars.FLOAT_MAX
#     total_loss = StaticVars.FLOAT_MAX

    def __init__(self, uid, iid, interactions, budget=1000, fobj=True, fconstraint=True):
        self.user_id = uid
        self.item_id = iid
        self.available_budget = budget

        self.satisfy_objective = fobj
        self.satisfy_contraints = fconstraint

        self.recommendation = None
        self.interactions = dict(original=interactions, initial=[], best=[])
        self.loss = dict(initial=StaticVars.FLOAT_MAX, best=StaticVars.FLOAT_MAX)
        self.iter_no = dict(initial=budget, best=budget, total=budget)
        self.budget_spent = dict(initial=budget, best=budget, total=budget)

        self.solution_found = False
        self.pos = StaticVars.INT_MAX
        self.cfs_dist = len(interactions)
        self.stats_per_cardinality = [0] * len(interactions)
        self.max_updated_card = -1

        self.len_interactions = len(self.interactions['original'])

    def __str__(self):
        sorted_recommended_items = [
            (n[0], n[1].detach().numpy().flatten()[0]) if isinstance(n[1], torch.Tensor)
            else (n[0], n[1]) for n in self.recommendation
        ]

        return (f'\n'
                f'user_id: {self.user_id}, item_id: {self.item_id}\n'
                f'yloss: {round(self.y_loss, 4)}, proximity_loss: {int(self.proximity_loss)}\n'
                f'Item {self.item_id} is in position {self.pos} now!!!\n'
                f'Found in iteration {self.iter_no["best"], {self.budget_spent}} and the interacted items are {self.interactions["best"]}\n'
                f'10-best recommended items {sorted_recommended_items}\n')

    def set_flags(self, do_objective, do_contraints):
        self.satisfy_objective = do_objective
        self.satisfy_contraints = do_contraints

    def needs_update(self, loss):
        if len(loss):
            does_contraints = (not self.satisfy_contraints or self.y_loss > loss['yloss'])
            does_objective = (not self.satisfy_objective or self.proximity_loss >= loss['proximity'])

            if does_contraints and does_objective: return True

        return False

    def set_values(self, predictions, interacted_items, tot_interacted_items, loss, iter_no, k=10):

        # get the ranking position of selected item in the list
        rk_data = st.rankdata(-predictions, method='ordinal')
        self.pos = rk_data[self.item_id]
#         self.recommends = sorted(enumerate(predictions), key=lambda x: x[1], reverse=True)[:k]
        accepted_preds = (rk_data <= k).nonzero()
        self.recommends = sorted(
            zip(predictions[accepted_preds], *accepted_preds), 
            key=lambda x: x[0], reverse=True)
        self.iter_found = iter_no
        self.y_loss = loss[0]
        self.proximity_loss = loss[1]
        self.interactions = interacted_items
        self.complete_interactions = tot_interacted_items

        self.solution_found = True

    def update_values(self, predictions, ranking, interacted_items, loss, iter_no, residual_budget, k):
        # self.pos <= ranking[self.item_id]
        if ranking[self.item_id] > k:

            if loss < self.loss['best']:
                # get the ranking position of selected item in the list
                # rk_data = st.rankdata(-predictions, method='ordinal')
                self.pos = ranking[self.item_id]
        #         self.recommends = sorted(enumerate(predictions), key=lambda x: x[1], reverse=True)[:k]
                accepted_preds = (ranking <= k).nonzero()
                self.recommendation = sorted(
                    zip(predictions[accepted_preds], *accepted_preds),
                    key=lambda x: x[0], reverse=True)

                self.iter_no['best'] = iter_no
                self.budget_spent['best'] = self.available_budget - residual_budget
                self.loss['best'] = loss
                self.interactions['best'] = interacted_items

                if not self.solution_found:
                    self.iter_no['initial'] = iter_no
                    self.budget_spent['initial'] = self.available_budget - residual_budget
                    self.loss['initial'] = loss
                    self.interactions['initial'] = interacted_items

                self.cfs_dist = self.len_interactions - len(self.interactions['best'])

                self.stats_per_cardinality[self.len_interactions - len(interacted_items) - 1] = max(
                    self.available_budget - residual_budget, self.stats_per_cardinality[len(interacted_items) - 1])

            self.solution_found = True

        self.iter_no['total'] = iter_no
        self.budget_spent['total'] = self.available_budget - residual_budget

In [7]:
class ComputeLoss:
    def __init__(self, target, original_input, top_k=10, weights=[1, 0, 0], total_CFs=1):
        self.target_item = target
        self.top_k = top_k
        self.original_items = original_input
        self.total_CFs = total_CFs
        (self.proximity_weight, self.diversity_weight, self.regularization_weight) = weights

    def _compute_yloss(self, target_score, kth_score):
        yloss = 0.0
        for i in range(self.total_CFs):
            temp_loss = max(0, target_score / kth_score - 1.0)
            # temp_loss = target_score / kth_score

            yloss += temp_loss
        return yloss / self.total_CFs

    def _compute_dist(self, x_hat, x1):
        """Compute weighted distance between two vectors."""
    #     return sum(abs(x_hat - x1))
#         diff = set(x1).difference(set(x_hat))
        diff = np.setdiff1d(x1, x_hat)
        return len(diff)

    def _compute_proximity_loss(self, cfs):
        proximity_loss = 0.0
        for i in range(self.total_CFs):
            proximity_loss += self._compute_dist(cfs, self.original_items)
        return proximity_loss / np.multiply(len(self.original_items), self.total_CFs)

    def _compute_diversity_loss(self):
        proximity_loss = 0.0
        return proximity_loss / self.total_CFs

    def _compute_regularization_loss(self, x):
        """Adds a linear equality constraints to the loss functions - to ensure all levels of a categorical variable sums to one"""
        regularization_loss = 0.0
        for i in range(self.total_CFs):
            pass
#             for v in self.encoded_categorical_feature_indexes:
#                 regularization_loss += torch.pow((torch.sum(self.cfs[i][v[0]:v[-1]+1]) - 1.0), 2)
#             regularization_loss += max(0, x - 1.0)

        return regularization_loss

    def compute_loss(self, cfs, preds, ranking, total_CFs=1):
        """Computes the overall loss"""

        yloss = self._compute_yloss(preds[self.target_item], preds[(ranking == self.top_k).nonzero()][0])
        proximity_loss = self._compute_proximity_loss(cfs) if self.proximity_weight > 0 else 0.0
        diversity_loss = self._compute_diversity_loss() if self.diversity_weight > 0 else 0.0
        regularization_loss = self._compute_regularization_loss(yloss) if self.regularization_weight > 0 else 0.0

        loss = yloss + (self.proximity_weight * proximity_loss) \
            - (self.diversity_weight * diversity_loss) \
            + (self.regularization_weight * regularization_loss)
        return loss

In [8]:
def find_cfs(dataset, model, excluded_item_pos, no_users=None, max_allowed_permutations=None, top_k=10, total_CFs=1):
    num_users = no_users or max(dataset.users_ids) + 1
    max_perms = max_allowed_permutations or dataset.max_sequence_length

    best_tot_loss_data = []
    best_yloss_data = []

    for user_id in trange(1, num_users):  # dataset.num_users):

        seq_size = len(dataset.sequences[dataset.user_ids==user_id])
        _total_loss = [None] * seq_size
        _yloss = [None] * seq_size

        for j in range(seq_size):    
            if all(v > 0 for v in dataset.sequences[dataset.user_ids==user_id][j]):    
                items_interacted = dataset.sequences[dataset.user_ids==user_id][j]
                predictions = -model.predict(items_interacted)
                predictions[items_interacted] = StaticVars.FLOAT_MAX

                kth_item = predictions.argsort()[top_k - 1]
                target_item = predictions.argsort()[min(top_k, int(excluded_item_pos)) - 1]

                _total_loss[j] = InteractionsInfo(user_id, target_item)
                _yloss[j] = InteractionsInfo(user_id, target_item, fobj=False)

                loss = ComputeLoss(target_item, items_interacted, top_k)

                counter = 1        

                for l in range(len(items_interacted) - 1, max(0, len(items_interacted) - max_perms), -1):
                    if _total_loss[j].solution_found: break

                    # produce permutations of various interactions
                    perm = combinations(items_interacted, l)

                    for i in perm:
                        # predict next top-k items about to be selected        
                        preds = model.predict(i)
                        
                        # convert logits produced by model, i.e., the probability distribution before normalization, 
                        # by using softmax
                        tensor = torch.from_numpy(preds).float()
                        preds = F.softmax(tensor, dim=0)

                        yloss = loss._compute_yloss(preds.numpy()[target_item], preds.numpy()[kth_item])
                        proximity_loss = loss._compute_proximity_loss(np.asarray(i)[np.newaxis, :])
                        
                        # keep info about the best solution found depending on an objective function
                        if _total_loss[j].needs_update(dict(yloss=yloss, proximity=proximity_loss)):                        
                            _total_loss[j].set_values(
                                preds, i, items_interacted, [yloss, proximity_loss], counter, top_k)
                            
#                         if _yloss[j].needs_update(dict(yloss=yloss, proximity=proximity_loss)):
#                             _yloss[j].set_values(
#                                 preds, i, items_interacted, [yloss, proximity_loss], counter, k)                 

                        counter += 1 

        best_tot_loss_data.append(_total_loss)
        best_yloss_data.append(_yloss)
        
    return (best_tot_loss_data, best_yloss_data)

In [17]:
from multiprocessing import Pool, cpu_count, RLock
from itertools import repeat


def _retrieve_solutions(params):
    user_id, d, m, sf, pos, init_budget, top_k, kwargs = params
#     tqdm_text = "#" + "{}".format(pid).zfill(3)

    _total_loss = []
    seq = d.sequences[d.user_ids == user_id]
    for j in range(min(1, len(seq))):  # seq_size):
        if all(v > 0 for v in seq[j]):
            items_interacted = seq[j].copy()
            predictions = -m.predict(items_interacted)
            predictions[items_interacted] = StaticVars.FLOAT_MAX

            target_item = predictions.argsort()[min(top_k, int(pos)) - 1]

            search_info = InteractionsInfo(user_id, target_item, items_interacted, init_budget)
            loss = ComputeLoss(target_item, items_interacted, top_k)
            strategy = sf(target_item, items_interacted, d.max_sequence_length, init_budget, m, **kwargs)

            counter = 1
            budget = strategy.get_init_budget()
            while budget > 0:
                perm, curr_budget = strategy.next_comb(reverse=search_info.solution_found)

                if perm is None: break  # there is no need to continue searching

                # predict next top-k items about to be selected
                preds = m.predict(perm)
                preds[perm] = -StaticVars.FLOAT_MAX
                # already taken care in strategy func, so do not count. 
                # We exec model again to retrieve useful info to store
#                 budget -= 1  # used Query

                # normalize logits produced by model, i.e., the probability distribution before normalization, 
                # by using softmax
#                             tensor = torch.from_numpy(preds).float()
# #                             tensor = F.softmax(tensor, dim=0)
#                             print('after', tensor, F.softmax(tensor, dim=-1), torch.max(tensor))

                rk_data = st.rankdata(-preds, method='ordinal')
                computed_loss = loss.compute_loss(perm, preds, rk_data)
#                             print('stats', user_id, computed_loss, len(perm), rk_data[target_item])

                # keep info about the best solution found depending on an objective function
                search_info.update_values(
                    preds, rk_data, perm, computed_loss, counter, curr_budget, top_k)

                if hasattr(strategy, 'set_score'):
                    reverse_search = strategy.set_score(
                        len(items_interacted) - len(perm) - 1,
                        preds[target_item],
                        preds[(rk_data == top_k).nonzero()][0]
                    )

                    if reverse_search:
                        _total_loss[j].solution_found = False
#                                     print('Forward Search applied!!!', len(items_interacted) - len(perm) - 1)

                strategy.reset_costs()
                counter += 1
        
                budget = curr_budget

            _total_loss.append(search_info)

    return _total_loss


def _find_cfs(dataset, model, strategy_func, target_item_pos, no_users=None, init_budget=1000,
              max_allowed_permutations=None, top_k=10, total_CFs=1, num_processes=10, **kwargs):

    print(f'The backend used is: {strategy_func.class_name}')

    num_users = no_users or max(dataset.users_ids) + 1
    best_tot_loss_data = dict.fromkeys(target_item_pos)

    with tqdm(total=len(target_item_pos), desc='target position loop') as pbar:
        for pos in target_item_pos:
            pbar.update(10)

            best_tot_loss_data[pos] = []
            for user_id in trange(1, num_users + 1, desc='users loop', leave=False):  # dataset.num_users):
    #                 best_tot_loss_data[pos].append(_total_loss)
                best_tot_loss_data[pos].extend(_retrieve_solutions((
                    user_id, dataset, model, strategy_func, pos, init_budget, top_k, kwargs)))


#         pool = Pool(processes=min(num_processes, cpu_count() - 1, 4), initargs=(RLock(),), initializer=tqdm.set_lock)
#         with Pool(processes=min(num_processes, cpu_count() - 1, 2), initializer=init, initargs=(l,)) as pool:
    #         jobs = [pool.apply_async(_retrieve_solutions, args=((n,dataset,model,strategy_func,pos,init_budget,top_k,kwargs),)) 
    #             for n in range(1, num_users + 1)]
#             jobs = list(pool.imap_unordered(_retrieve_solutions, zip(
#                 range(1, num_users + 1), repeat(dataset), repeat(model), repeat(strategy_func),
#                             repeat(pos), repeat(init_budget), repeat(top_k), repeat(kwargs)
#                         )))
#             best_tot_loss_data[pos] = [jobs[i].get() for i in trange(len(jobs))]

#         with Pool(processes=min(num_processes, cpu_count() - 1, 2)) as p:
# #             best_tot_loss_data[pos].append(list(tqdm(p.imap_unordered(
# #                 _retrieve_solutions, zip(
# #                     range(1, num_users + 1), repeat(dataset), repeat(model), repeat(strategy_func),
# #                     repeat(pos), repeat(init_budget), repeat(top_k), repeat(kwargs)
# #                 )), total=num_users, leave=False)
# #             ))
#             list(tqdm(p.imap_unordered(
#                 _retrieve_solutions, zip(
#                     range(1, num_users + 1), repeat(dataset), repeat(model), repeat(strategy_func),
#                     repeat(pos), repeat(init_budget), repeat(top_k), repeat(kwargs)
#                 )), total=num_users))
# #                 r = list(tqdm(p.imap(_foo, range(30)), total=30))

    return best_tot_loss_data

In [10]:
def convert_res_to_lists(cfs, cnt, non_achieved_target, technique):
    for key, values in cfs.items():
        total_data = []
        cnt[key].setdefault(technique, [])
        cfs_no = 0

#         for items in values:
        for rec in values:
            if rec is None: continue

#                 if not rec.solution_found or rec.pos < 10:
#                     non_achieved_target[key].append(rec.user_id)
#                     continue

            total_data.append([
                len(rec.interactions['original']) - len(rec.interactions['initial']), rec.cfs_dist,
                # for boxplot
                rec.budget_spent['initial'], rec.budget_spent['best'],
                rec.iter_no['initial'], rec.iter_no['best'],
                rec.user_id, len(rec.interactions['original'])
            ] + rec.stats_per_cardinality)

            cfs_no = len(rec.interactions['original'])

        cnt[key][technique].append(Counter(item[0] for item in total_data))
        cnt[key][technique].append(Counter(item[1] for item in total_data))
        cnt[key][technique].append([item[2] for item in total_data])
        cnt[key][technique].append([item[3] for item in total_data])
        cnt[key][technique].append([item[4] for item in total_data])
        cnt[key][technique].append([item[5] for item in total_data])
        cnt[key][technique].append([item[6] for item in total_data])
        cnt[key][technique].append([item[7] for item in total_data])
        cnt[key][technique].append([item[1] for item in total_data])

        for i in range(cfs_no):
            cnt[key][technique].append([item[8 + i] for item in total_data])

    return cnt, non_achieved_target

In [11]:
def gpu_embeddings_to_cosine_similarity_matrix(E):
    """ 
    Converts a tensor of n embeddings to an (n, n) tensor of similarities.
    """
    dot = E @ E.t()
    norm = torch.norm(E, 2, 1)
    x = torch.div(dot, norm)
    x = torch.div(x, torch.unsqueeze(norm, -1))
    return x

In [12]:
from torch.nn.functional import cosine_similarity


def embeddings_to_cosine_similarity_matrix(E):
    """ 
    Converts a a tensor of n embeddings to an (n, n) tensor of similarities.
    """
    similarities = [[cosine_similarity(a, b, dim=0) for a in E] for b in E]
#     similarities = list(map(torch.cat, similarities))
    similarities = list(map(lambda x: torch.stack(x, dim=-1), similarities))
    return torch.stack(similarities)

In [13]:
from scipy.spatial.distance import pdist, squareform


def compute_sim_matrix(dataset, metric='jaccard', adjusted=False):
    # compute the item-item similarity matrix utilizing implicit feedback,
    # i.e., whether interacted or not with an item

    M = np.zeros((dataset.num_users, dataset.num_items), dtype=np.bool)
    for u in trange(1, dataset.num_users):
        np.add.at(
            M[u], dataset.item_ids[dataset.user_ids == u],
            dataset.ratings[dataset.user_ids == u]
        )

    if adjusted:
        M_u = M.mean(axis=1)
        M = M - M_u[:, np.newaxis]

    similarity_matrix = 1 - squareform(pdist(M.T, metric))

    return similarity_matrix

In [14]:
from collections import Counter


def rank_interactions_to_excluded_item_per_user(cfs, sims_matrix):
    non_solvable_cases = []
    total_data = []

    for items in cfs:
        for rec in items:
            if rec is None: continue

            if not rec.solution_found:
                non_solvable_cases.append(rec.user_id)
                continue

            items_rank = st.rankdata(sims_matrix[rec.item_id, rec.complete_interactions])
            similarity_rank = len(rec.complete_interactions) - items_rank + 1
            del_items_indices = np.where(np.isin(
                rec.complete_interactions, 
                list(set(rec.complete_interactions).difference(set(rec.interactions)))
            ))
            total_data.extend(sorted(similarity_rank[del_items_indices].astype(int)[-1:]))

    return (Counter(total_data), non_solvable_cases)

In [18]:
# A simple class stack that only allows pop and push operations
class Stack:

    def __init__(self):
        self.stack = []

    def pop(self):
        if len(self.stack) < 1:
            return None
        return self.stack.pop()

    def push(self, item):
        self.stack.append(item)

    def size(self):
        return len(self.stack)


# And a queue that only has enqueue and dequeue operations
class Queue:

    def __init__(self):
        self.queue = []

    def enqueue(self, item):
        self.queue.append(item)

    def dequeue(self):
        if len(self.queue) < 1:
            return None
        return self.queue.pop(0)

    def size(self):
        return len(self.queue)

    def clear(self):
        del self.queue[:]

    def get(self, i):
        return self.queue[i]
    
    def setter(self, i, v):
        self.queue[i] = v